In [163]:
''' The Jupyter notebook is a web-based notebook environment for interactive computing. '''
print(f"ssl verification via Search-Guard")

ssl verification via Search-Guard


In [ ]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import dotenv
import socket
import requests
import sys
import warnings
import logging
from requests.auth import HTTPBasicAuth
from IPython.display import HTML
from cryptography import x509
import datetime
from cryptography.hazmat.backends import default_backend
warnings.filterwarnings("ignore")

In [ ]:
''' pip install python-dotenv'''
# load_dotenv() # will search for .env file in local folder and load variables
# Reload the variables from your .env file, overriding existing ones
dotenv.load_dotenv(".env", override=True)

True

In [ ]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [ ]:
logger = logging.getLogger("api-call")

In [166]:
INDEX = "om_test"

In [167]:
ca_cert_path = '../upgrade-script/certs/qa13_new/test.pem'

In [ ]:
def get_certificate_info(path):
    with open(path, "rb") as f:
        cert_data = f.read()
    
    cert = x509.load_pem_x509_certificate(cert_data, default_backend())

    print(f"Certificate subject: {cert.subject}")
    
    # Accessing validity period
    not_before = cert.not_valid_before_utc
    not_after = cert.not_valid_after_utc
    
    print(f"Certificate valid from: {not_before}")
    print(f"Certificate valid until: {not_after}")

    # Check if currently valid
    now = datetime.datetime.now(datetime.timezone.utc)
    if not_before <= now <= not_after:
        print("Certificate is currently valid.")
    else:
        print("Certificate is not currently valid.")

In [168]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized rquests. Insufficient permissions',
    403 : 'Unauthorized request. Insufficient permissions. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [169]:
''' base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [170]:
''' base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [171]:
def get_headers(auth):
    ''' Elasticsearch Header '''
    ''' 
    Basic Authentication is a method for an HTTP user agent (e.g., a web browser) 
    to provide a username and password when making a request. 
    You can send the authorization header 
    when making requests and accessing to ES Cluster based on Search-Guard as X-pack. 
      
    Basic Auth : 
    {
        'Content-type': 'application/json', 
        'Authorization' : 'Basic base64.encode(id:password), 
        'Connection': 'close'
    }
    '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv(auth)),
            'Connection': 'close'
    }

In [172]:
source_es_host = "http://localhost:9202"
# source_es_host = os.getenv('ES_DEV_V8_HOST')

### Verify if user can be accessed to the ES cluster with Search Guard as X-pack
* ES Version : ES v.8.17.0 via https protocal
* X-pack : Search Guard for ES v.8.17.0
* Library : ES cliient (https://elasticsearch-py.readthedocs.io/en/v8.17.0/)
* Pass the link to the certificate for ssl validation with user credential via get_es_instance function

In [ ]:
def get_es_instance(auth):
    ''' create es instance with user credential and ssl verification '''
    try:
        ''' To disable certificate verification, at the client side, one can use verify attribute.  -- Verify False''' 
        # es_client = Elasticsearch(hosts="{}".format(source_es_host), headers=get_headers(), timeout=5,  verify_certs=False)
        
        ''' We can also pass the link to the certificate for validation via python requests only. '''
        ''' If we don't pass the link for the certification, we may get the message like 
        caused by: SSLError([SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)) if it doesn't exist ca certs
        '''
        es_client = Elasticsearch(
                                  hosts="{}".format(source_es_host), 
                                  # hosts=['https://localhost1:9200','https://localhost2:9200','https://localhost3:9200'],
                                  use_ssl=True, 
                                  # ca_certs=ca_cert_path, 
                                  headers=get_headers(auth), 
                                  timeout=5,  
                                  # verify_certs=True
                                  verify_certs=False
                                 )
        return es_client
    except Exception as e:
        print(str(e))


In [174]:
es_client_wx = get_es_instance('BASIC_AUTH_WX')

In [ ]:
try:
    print(es_client_wx.cat.health())
    # print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
    es_health_dict = es_client_wx.cluster.health()
    print(json.dumps(es_health_dict, indent=2))
    
    es_user_es_indices_list = list(es_client_wx.indices.get("*").keys())
    # print(es_wxusers_es_indices_list)
    print("** A list of ES Indices for users **")
    print(json.dumps(es_user_es_indices_list, indent=2))
except Exception as e:
    print(str(e))

ConnectionError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)) caused by: SSLError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006))


In [176]:
es_client_om = get_es_instance('BASIC_AUTH_OM')

In [ ]:
try:
    print(es_client_om.cat.health())
    # print(json.dumps(es_client.cat.indices(params={"format": "json"}), indent=2))
    es_health_dict = es_client_om.cluster.health()
    print(json.dumps(es_health_dict, indent=2))
    
    es_user_es_indices_list = list(es_client_wx.indices.get("*").keys())
    # print(es_wxusers_es_indices_list)
    print("** A list of ES Indices for users **")
    print(json.dumps(es_user_es_indices_list, indent=2))
except Exception as e:
        print(str(e))

ConnectionError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)) caused by: SSLError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006))


### Verify if user can be accessed to the ES cluster with Search Guard as X-pack
* ES Version : ES v.8.17.0 via https protocal
* X-pack : Search Guard for ES v.8.17.0
* Library : Request Package
* Pass the link to the certificate for ssl validation  with user credential via get_es_instance function

In [178]:
def request_verify_es_health(auth):
    try:
        ''' wxuser '''
        res = requests.get(
            url=source_es_host, 
            headers=get_headers(auth), 
            verify=ca_cert_path)
    
        if not res.status_code == 200:
           print(res.status_code, http_status_code.get(res.status_code, "None"))
           print('\n')
           return
           
        print(json.dumps(res.json(), indent=2))
        print('\n')
    except Exception as e:
        print(str(e))

In [179]:
request_verify_es_health('BASIC_AUTH_WX')

{
  "name": "fn-dm-bees-omni-data-01",
  "cluster_name": "docker-elasticsearch",
  "cluster_uuid": "w3idAZFFS1uh6v5LieHUxA",
  "version": {
    "number": "8.8.0",
    "build_flavor": "default",
    "build_type": "docker",
    "build_hash": "c01029875a091076ed42cdb3a41c10b1a9a5a20f",
    "build_date": "2023-05-23T17:16:07.179039820Z",
    "build_snapshot": false,
    "lucene_version": "9.6.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}




In [180]:
request_verify_es_health('BASIC_AUTH_OM')

{
  "name": "fn-dm-bees-omni-data-01",
  "cluster_name": "docker-elasticsearch",
  "cluster_uuid": "w3idAZFFS1uh6v5LieHUxA",
  "version": {
    "number": "8.8.0",
    "build_flavor": "default",
    "build_type": "docker",
    "build_hash": "c01029875a091076ed42cdb3a41c10b1a9a5a20f",
    "build_date": "2023-05-23T17:16:07.179039820Z",
    "build_snapshot": false,
    "lucene_version": "9.6.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}




In [ ]:
HTML('<pre style="background-color: #A52A2A; color: white;">Job is being performed correctly</pre>')